# NRC-09 Darks: Readnoise and 1/f Noise

|                                |                                                          |
|:---                            |:---                                                      |
|__CAR Title__                   | NRC-09: Darks                                            |
|__APT Program #__               | 1062                                                     |
|__NGAS #__                      | 102                                                      |
|__CAR Execution Dates(s) (UT):__| TBD                                                      |
|__JIRA Ticket Links:__          | CAR [NRCCOM-13](https://jira.stsci.edu/browse/NRCCOM-13) ||
|                                | CAP [NRCCOM-14](https://jira.stsci.edu/browse/NRCCOM-14) ||
|                                | [JSOCOPS-15](https://jira.stsci.edu/browse/JSOCOPS-15)   ||
|                                | NRCC1C-mm TBD                                            ||
|                                | NRCC1C-nn TBD                                            |
|__Analysis Team/Roles:__        | Leads: Karl Misselt (IDT), Alicia Canipe (IST)           ||
|                                | Jarron Leisenring (Analysis/Scripts)                     ||
|                                | Thomas Beatty (TSO expertise)                            ||
|                                | Bryan Hilbert (Analysis/Scripts)                         ||
|                                | Ben Sunnquist (Analysis/Scripts)                         ||
|                                | Armin Rest (Analysis/Scripts)                            ||

## Table of Contents

* [Objective](#objective)
* [Relevant links and documentation](#links)
* [Environment for analysis](#environment)
* [Imports](#imports)
* [Data for analysis](#data)
* [Readnoise](#readnoise)
* [1/f noise](#readnoise)
* [Reference pixel stability](#refpix)

<a id='objective'></a>
## Objective

Characterize the properties (stability, spatial structure, etc.) of the readnoise and 1/f noise using the methods described in Rest 2014 (JWST-STScI-004118). Derive readnoise images for all detectors and compare them with ground test exposures. Note: we should think about reference pixel stability characterization - some reference pixels were unstable when they did the tuning. Compare with ground test data. 

<a id='links'></a>
## Relevant links and documentation

|                       |                                                                                  |
|:---                   |:---                                                                              |
__JWQL dark monitor__   |                                                                             |
__NRC-09 CAR page__     |[Review Notes NRC-09](https://outerspace.stsci.edu/display/JN/Review+Notes+NRC-09)|
__NRC-09 CAP page__     |[CAP: NIRCam-09](https://outerspace.stsci.edu/display/JN/CAP%3A+NIRCam-09)        |
__Scripts__             |[NIRCam Commissioning Analysis Scripts](https://outerspace.stsci.edu/display/JN/NIRCam+Commissioning+Analysis+Scripts)                            |

<a id='environment'></a>
## Environment for analysis

Follow instructions for downloading the latest version of the pipeline to get the necessary analysis tools. Activate your environment, and then add additional tools. For this analysis, you will also need the readnoise algorithm in the jwst_reffiles package, which is used to generate readnoise reference files. 

Note that pipeline software is not guaranteed to work on Windows machines, but it *should* work, in theory. 

[Pipeline installation](https://github.com/spacetelescope/jwst#installing-latest-releases)

```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst
```

or for a specific version:
```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst==1.2.3
```

and add additional tools used that are not included in the ```jwst``` package:
```python
pip install ipython jupyter matplotlib pylint pandas
```

and install [jwst_reffiles](https://jwst-reffiles.readthedocs.io/en/latest/official_bad_pixel_mask.html?highlight=hot%20pixel):
```python
pip install git+https://github.com/spacetelescope/jwst_reffiles.git
```

<a id='imports'></a>
## Imports

In [ ]:
import argparse
import requests
import yaml
from glob import glob
from jwst import datamodels
from jwst.datamodels import dqflags
import numpy as np
import pandas as pd
from astropy.io import fits, ascii
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch
from jwst_reffiles.readnoise import readnoise
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import FuncFormatter, MaxNLocator

%matplotlib inline

In [ ]:
def show_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None):
    """Hilbert's function to generate a 2D, log-scaled image of the data, 
    with an option to highlight a specific pixel (with a red dot).
    
    Parameters
    ----------
    data_2d : numpy.ndarray
        Image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    xpixel : int
        X-coordinate of pixel to highlight
        
    ypixel : int
        Y-coordinate of pixel to highlight
        
    title : str
        String to use for the plot title
    """
    norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                          stretch=LogStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    fig.colorbar(im, label='DN')
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

<a id='data'></a>
## Data for analysis

Data will be downloaded from MAST using the code written by Mario Gennaro and Armin Rest, and stored in our NIRCam data location: **TBD**

In [ ]:
# base_dir = '/ifs/jwst/wit/nircam/'
base_dir = '/ifs/jwst/wit/witserv/data7/nrc/'
analysis_dir = './'
ground = 'otis_long_darks'
flight = 'TBD'

In [ ]:
# files = glob(base_dir+ground+'/NRC*_484_*uncal.fits')
files = glob("*linearitystep.fits")

<a id='readnoise'></a>
## Readnoise

[Visit the documentation](https://jwst-reffiles.readthedocs.io/en/latest/api.html?highlight=superbias#module-jwst_reffiles.readnoise.readnoise) for more information on the readnoise algorithm. This module creates a readnoise reference file that can be used in the JWST calibration pipeline. Two different methods can be used:

Method 1 (method=’stack’): Create CDS images for each input ramp. Stack all of these images from each input ramp together. The readnoise is the sigma-clipped standard deviation through this master CDS image stack.

Method 2 (method=’ramp’): Calculate the readnoise in each ramp individually, and then average these readnoise images together to produce a final readnoise map.

We will use these tools to investigate the readnoise. Details about the arguments for the tools are included in the documentation. 

In [ ]:
output_filename = 'readnoise_jwst_reffiles.fits'

Choose a method to use. Repeat the exercise with the other method if necessary. 

In [ ]:
choose_method = 'stack'

In [ ]:
readnoise.make_readnoise(files, method=choose_method, group_diff_type='independent', 
                         clipping_sigma=3.0, max_clipping_iters=3, nproc=1, slice_width=50, 
                         single_value=False, outfile=output_filename, 
                         author='jwst_reffiles', description='CDS Noise Image', pedigree='GROUND', 
                         useafter='2015-10-01T00:00:00', history='', subarray=None, readpatt=None, 
                         save_tmp=False)

### Readnoise image 

In [ ]:
readnoise = datamodels.ReadnoiseModel(output_filename)

In [ ]:
show_image(readnoise.data, vmin=5, vmax=100)

<a id='1f'></a>
## 1/f noise

Add Armin's scripts. Jarron has a script that takes the power spectrum of long darks and decomposes it into a variety of white noise, 1/f noise (correlated between the 4 channels), and 1/f noise that is amplifier-dependent

<a id='refpix'></a>
## Reference pixel stability

Add Jarron's script that looks at how stable reference pixels are versus how stable the active pixels are to get an idea of any instability between the two.